In [1]:
#niezbędne biblioteki
import gym
import random
import numpy as np
import tensorflow as tf
from keras.models     import Sequential
from keras.layers     import Dense, Conv2D, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
from skimage import color

Using TensorFlow backend.


In [2]:
env = gym.make('Enduro-v0')
env.reset()
goal_steps = 5000
#score_requirement = 6000
intial_games = 100

In [3]:
#do zobaczenia informacji jakie zwraca nam gra
def play_a_random_game_first():
    for step_index in range(goal_steps):
        #env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("done: {}".format(done))
        if done:
            break
    env.reset()

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [5]:
#funkcja do preprocesingu ramek. Ucinam górną część (niesitotne niebo) i zmieniam na czarno-biały obrazek
def preprocces_frame(frame):
    img = color.rgb2gray(frame)
    img=img[50:,:]
    img=img.reshape(img.shape[0], img.shape[1],-1)
    return img

In [6]:
#generowanie danych. Zapisuje tylko ścieżki, które dały dobry rezultat
def model_data_preparation(require_score):
    training_data = []
    observations=[]
    actions=[]
    i=0
    j=0
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(4300):
            action = random.randrange(0, 9)
            observation, reward, done, info = env.step(action)
        for step_index in range(goal_steps):
            action = random.randrange(0, 9)
            observation, reward, done, info = env.step(action)
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += 1
            if done:
                break
        if score >= require_score:
            print("Udało się", j)
            j+=1
            for data in game_memory:
                data[1]=np_utils.to_categorical(data[1],9).reshape(9)
                data[0]= preprocces_frame(data[0])
                observations.append(data[0])
                actions.append(data[1])

        env.reset()
        print(i)
        i+=1
    
    return [observations, actions]


In [7]:
training_data = model_data_preparation(160)

Udało się 0
0
1
Udało się 1
2
Udało się 2
3
4
5
6
7
8
9
Udało się 3
10
11
12
13
14
Udało się 4
15
16
17
Udało się 5
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Udało się 6
44
45
46
47
48
49
50
51
Udało się 7
52
53
Udało się 8
54
55
56
Udało się 9
57
58
59
60
61
Udało się 10
62
Udało się 11
63
64
65
Udało się 12
66
67
68
69
70
71
72
73
74
75
Udało się 13
76
77
78
79
80
81
82
83
84
85
86
87
Udało się 14
88
Udało się 15
89
Udało się 16
90
91
92
93
94
95
96
97
98
99


In [8]:
#Funkcja do budowania modelu: prawdopodobnie będę generował i uczył model jednocześnie, by oszczędzić pamięć
def build_model():
    model = Sequential()
    model.add(Conv2D(30, (3,3), input_shape=(160,160, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(9, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())

    return model


In [9]:
def train_model(training_data):
    X = np.array(training_data[0])
    y = np.array(training_data[1])
    model = build_model()
    
    model.fit(X, y, epochs=10)
    return model

In [10]:
trained_model = train_model(training_data)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2846/2846 [==============================] - 31s 11ms/step - loss: 0.3505
Epoch 2/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0984
Epoch 3/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0934
Epoch 4/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0884
Epoch 5/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0844
Epoch 6/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0797
Epoch 7/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0753
Epoch 8/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0703
Epoch 9/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0663
Epoch 10/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0619


In [11]:
#Testowanie
scores = []
#choices = []
for each_game in range(5):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, prev_obs.shape[0], prev_obs.shape[1],
                                                                      prev_obs.shape[2])))
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = preprocces_frame(new_observation)
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))
#print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[4464, 4408, 4444, 4397, 4435]
Average Score: 4429.6


In [12]:
#Dla porównania losowe ruchy
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        action = random.randrange(0,8)
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))

[4464, 4425, 4404, 4410, 4443, 4429, 4433, 4406, 4459, 4447, 4414, 4438, 4484, 4421, 4426, 4438, 4447, 4452, 4463, 4453]
Average Score: 4437.8
